In [1]:
import os
import json
from dotenv import find_dotenv, load_dotenv
import openai
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [2]:
#openai.api_key = st.secrets["OPENAI_API_KEY"]
load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

# Instantiate GPT Model 
model = ChatOpenAI(model="gpt-4-turbo")

In [3]:
# Adapt the prompt folder path
PROMPT_FOLDER_PATH = os.path.join(os.getcwd(), 'prompts')
assert os.path.exists(PROMPT_FOLDER_PATH)

AssertionError: 

In [ ]:
def load_prompts(prompt_folder_path):
    with open(os.path.join(prompt_folder_path, "system_prompt.txt"), "r") as f:
        system_prompt = f.read()
    with open(os.path.join(prompt_folder_path, "human_prompt.txt"), "r") as f:
        human_prompt = f.read()
    return system_prompt, human_prompt


copywriting_system_prompt, copywriting_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "Copywriting"))
brand_review_system_prompt, brand_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "BrandReview"))
copywriting_review_system_prompt, copywriting_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "CopywritingReview"))
tov_review_system_prompt, tov_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "TOVReview"))
editor_system_prompt, editor_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "Editor"))
role =  open(os.path.join(PROMPT_FOLDER_PATH, "Role", "role.txt"), "r").read()

In [4]:
def generate_structured_content(hotel_data, language):
    
    # Output parser creation
    class Copywriting(BaseModel):
        generated_text: str = Field(description="The written product detail page.")

    class Feedback(BaseModel):
        feedback: str = Field(description="The feedback on the copy.")

    class Edition(BaseModel):
        edited_text: str = Field(description="The edited and improved version of the product detail page.")

    copywriting_output_parser = JsonOutputParser(pydantic_object=Copywriting)
    review_output_parser = JsonOutputParser(pydantic_object=Feedback)
    edition_output_parser = JsonOutputParser(pydantic_object=Edition)

    # Chains
    ## Copywriting
    copywriting_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(copywriting_system_prompt),
            HumanMessagePromptTemplate.from_template(copywriting_human_prompt)
        ],
        input_variables=["role", "persona", "language", "product_data", "brand_knowledge", "copywriting_guidelines", "platform_specs"],
        partial_variables={"format_instructions": copywriting_output_parser.get_format_instructions()}
    )

    copywriting_chain = copywriting_prompt | model | copywriting_output_parser

    ## Brand Review
    brand_review_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(brand_review_system_prompt),
            HumanMessagePromptTemplate.from_template(brand_review_human_prompt)
        ],
        input_variables=["role", "brand_knowledge", "generated_text"],
        partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
    )
    brand_review_chain = brand_review_prompt | model | review_output_parser

    ## Copywriting Review
    copywriting_review_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(copywriting_review_system_prompt),
            HumanMessagePromptTemplate.from_template(copywriting_human_prompt)
        ],
        input_variables=["role", "copywriting_guidelines", "generated_text"],
        partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
    )
    copywriting_review_chain = copywriting_review_prompt | model | review_output_parser

    ## TOV Review 
    
    # TODO : THIS IS ALREADY HANDLED
    def format_examples(example_dict):
        formated_text = ""
        for product in example_dict.keys():
            formated_text += str(product.upper()) + "\n"
            formated_text +=  str(example_dict[product]) + "\n"
            formated_text += "-" * 10 + "\n"
        return formated_text
            
    tov_review_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(tov_review_system_prompt),
            HumanMessagePromptTemplate.from_template(tov_review_human_prompt)
        ],
        input_variables=["role", "reference_examples", "generated_text"],
        partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
    )
    tov_review_chain = tov_review_prompt | model | review_output_parser

    ## Edition
    edition_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(editor_system_prompt),
            HumanMessagePromptTemplate.from_template(editor_human_prompt)
        ],
        input_variables=["role", "perona", "generated_text", "feedback", "brand_knowledge", "copywriting_guidelines", "reference_examples", "platform_specs"], 
        partial_variables={"format_instructions": edition_output_parser.get_format_instructions()}
    )
    edition_chain = edition_prompt | model | edition_output_parser
    
    def format_feedback(reviews):
        brand_review, copywriting_review, tov_review = reviews
        return f"""Brand Feedback:
    {brand_review}

    Copywriting Feedback:
    {copywriting_review}

    Tone of Voice Feedback:
    {tov_review}

    """

    write_product_description_chain = (
        RunnablePassthrough.assign(generated_text=copywriting_chain)
        | RunnablePassthrough.assign(reference_examples=itemgetter("reference_examples") | RunnableLambda(format_examples))
        | RunnablePassthrough.assign(
            brand_review=brand_review_chain | itemgetter("feedback"),
            copywriting_review=copywriting_review_chain | itemgetter("feedback"),
            tov_review=tov_review_chain | itemgetter("feedback")
        ) | RunnablePassthrough.assign(feedback=itemgetter("brand_review", "copywriting_review", "tov_review") | RunnableLambda(format_feedback))
        | RunnablePassthrough.assign(edited_text=edition_chain | itemgetter("edited_text"))
        | itemgetter("edited_text")
    )   
    
    # TODO ADAPT
    if product_name in product_data[language]:
        product_details = product_data[language][product_name]
    else:
        def format_product_details(product_detail_list):
            return  f"PRODUCT DESCRIPTION:\n{product_detail_list[0]}\n\nOBJECT DESCRIPTION:\n{product_detail_list[1]}\n\nADDITIONAL INFO:\n{product_detail_list[2]}"
        product_details = format_product_details(product_details)

            
    return write_product_description_chain.invoke({
        "role": role, 
        "persona": persona, 
        "language": language,
        "product_name": product_name,
        "product_data": product_details,
        "brand_knowledge": brand_knowledge,
        "copywriting_guidelines": copywriting_guidelines,
        "reference_examples": product_data[language],
        "platform_specs": hermes_specs,   
        "existing_product_dp": product_data[language][product_name] 
    })